<a href="https://colab.research.google.com/github/jeaneigsi/cookbook/blob/main/reftraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU transformers  bitsandbytes

In [2]:
!pip install torch==2.2.1

In [3]:
!pip install git+https://github.com/frankaging/pyreft.git

  Cloning https://github.com/frankaging/pyreft.git to /tmp/pip-req-build-_89rjsw1
  Running command git clone --filter=blob:none --quiet https://github.com/frankaging/pyreft.git /tmp/pip-req-build-_89rjsw1
  Resolved https://github.com/frankaging/pyreft.git to commit fd7d6ef2ae96a7b9519a50d7181e2ce88a5c68bd
  Preparing metadata (setup.py) ... done


In [4]:
!pip install -U accelerate

In [5]:
!pip install evaluate

  Using cached evaluate-0.4.1-py3-none-any.whl (84 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)


In [6]:
%cd

/root


In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,TrainingArguments,BitsAndBytesConfig

from pyreft import (
    get_reft_model,
    ReftConfig,
    ConsreftIntervention
)

In [13]:
model="mistralai/Mistral-7B-Instruct-v0.2"

quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_dtype="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [9]:
model=AutoModelForCausalLM.from_pretrained(model,device_map="auto", quantization_config=quantization_config)



config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [23]:
model_base=AutoModelForCausalLM.from_pretrained(model,torch_dtype=torch.bfloat16, device_map='cuda')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
print(model_base)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [20]:
reft_config = ReftConfig(representations={"layer": 15, "component": "block_output",
    "intervention": ConsreftIntervention(
    embed_dim=model_base.config.hidden_size, low_rank_dimension=1)})



In [24]:

reft_model = get_reft_model(model_base, reft_config)
reft_model.print_trainable_parameters()

trainable intervention params: 4,097 || trainable model params: 0
model params: 7,241,732,096 || trainable%: 5.657486283237397e-05


In [27]:
 tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",model_max_length=2048, padding_side="right", use_fast=False)
tokenizer.pad_token = tokenizer.unk_token


In [28]:
def tokenize(prompt):
    result=tokenizer(
    prompt,
    truncation=True,
    max_length=512,
    padding="max_length",
    )
    result['labels']=result['input_ids'].copy()
    return result

In [32]:
!pip install datasets==2.16.0

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [36]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [34]:
import locale

In [37]:
!pip install datasets==2.16.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Found existing installation: datasets 2.18.0
    Uninstalling datasets-2.18.0:
      Successfully uninstalled datasets-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This b

In [ ]:
from datasets import load_dataset

datasets=load_dataset()

In [31]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to('cuda')
model_base.to('cuda')

generated_ids = model_base.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Certainly! Here's a simple and classic mayonnaise recipe that you can try at home.

Ingredients:
- 1 egg yolk
- 1 tablespoon of mustard (Dijon or yellow)
- 1 cup of vegetable oil
- 1-2 tablespoons of white wine vinegar or fresh lemon juice
- Salt, to taste

Instructions:
1. In a large bowl, whisk together the egg yolk and mustard until they're well combined.
2. Very slowly, drop by drop, begin to drizzle in the vegetable oil, continuing to whisk constantly.
3. As the mixture begins to thicken and emulsify, you can gradually add the oil in a thin stream, whisking all the while.
4. Once all the oil has been incorporated, whisk in the vinegar or lemon juice and a pinch of salt, to taste.
5. Cover the bowl with plastic wrap, pressing it 